In [ ]:
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, RandomFlip, RandomRotation, RandomZoom, BatchNormalization, GaussianNoise, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from livelossplot import PlotLossesKeras

In [ ]:
# show gpu availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# set gpu memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        pass

In [ ]:
# Khai báo các thông số
img_width, img_height = 180, 180
batch_size = 64
epochs = 25
num_classes = 3
input_shape = (img_width, img_height, 3)

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = '../dataset_without_bmi'
dataset = 'dataset_without_bmi'

# remove .DS_Store file in data_dir
os.system('find ' + data_dir + ' -name ".DS_Store" -delete')

# Danh sách các lớp (classes) là các thư mục con trong thư mục chứa dữ liệu
classes = os.listdir(data_dir)
print(classes)

In [ ]:
# Tạo empty lists để lưu trữ đường dẫn hình ảnh và nhãn tương ứng
image_paths = []
labels = []

# Lặp qua mỗi lớp
for class_label in classes:
  class_dir = os.path.join(data_dir, class_label)

  # Lặp qua mỗi hình ảnh trong mỗi lớp
  for image_name in os.listdir(class_dir):
    image_path = os.path.join(class_dir, image_name)
    image_paths.append(image_path)
    labels.append(class_label)

# Chia dữ liệu thành tập huấn luyện và tập validation
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Tạo DataFrame cho tập huấn luyện
train_dataframe = pd.DataFrame({'image_path': train_image_paths, 'label': train_labels})
# Tạo DataFrame cho tập validation
val_dataframe = pd.DataFrame({'image_path': val_image_paths, 'label': val_labels})

In [ ]:
def grayscale(image):
    return tf.image.rgb_to_grayscale(image)

# Tạo ImageDataGenerator cho tập huấn luyện
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa lại giá trị pixel về khoảng [0,1]
    preprocessing_function=grayscale,  # Chuyển ảnh màu sang ảnh xám
)

# Tạo ImageDataGenerator cho tập validation, chỉ cần rescale lại giá trị pixel
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Tạo generator cho tập huấn luyện
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_dataframe,  # DataFrame chứa đường dẫn hình ảnh và nhãn cho tập huấn luyện
    x_col="image_path",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)

# Tạo generator cho tập validation
validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_dataframe,  # DataFrame chứa đường dẫn hình ảnh và nhãn cho tập validation
    x_col="image_path",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)

# plot a batch of images and labels
import matplotlib.pyplot as plt
x_batch, y_batch = next(train_generator)
plt.figure(figsize=(12, 9))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(x_batch[i])
    plt.title(classes[y_batch[i].argmax()])
    plt.axis('off')
plt.show()


In [ ]:
# Load pre-trained ResNet50 model without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
data_augmentation = Sequential(
  [
    RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    RandomRotation(0.1),
    RandomZoom(0.1),
  ]
)

In [ ]:
# Add custom layers to avoid overfitting
model = Sequential()
model.add(data_augmentation)
model.add(base_model)
model.add(BatchNormalization())
model.add(GaussianNoise(0.01))
model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), bias_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), bias_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Freeze convolutional base
base_model.trainable = False

In [ ]:
# Compile model
model.compile(optimizer=Adamax(learning_rate=0.001), loss=CategoricalCrossentropy(), metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[PlotLossesKeras(), EarlyStopping(monitor='val_loss', patience=5)]
)

In [ ]:
# Evaluate the model using f1 score, precision, recall, accuracy
y_pred = model.predict(validation_generator)
y_pred = y_pred.argmax(axis=1)
y_true = validation_generator.classes
f1 = f1_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)
print('f1 score: ', f1)
print('precision: ', precision)
print('recall: ', recall)
print('accuracy: ', accuracy)